## EDA

Para este notebook he decidido trabajar con un kernel que tengo el cual cree con anaconda y lo llamé mlops, podés añadir las dependencias que necesites al toml si lo deseas e importar el kernel en el notebook. En lo personal, cuando son cosas muy de EDA, prefiero trabajar local y en ambientes que tengo especificamente para eso.  

In [1]:
!pip install wandb -qq

In [26]:
from fastai.vision.all import *
import wandb
from params import *
import numpy as np

In [33]:

DEBUG = False # set this flag to True to use a small subset of data for testing

In [11]:
URL = "https://storage.googleapis.com/wandb_course/bdd_simple_1k.zip"

In [12]:
path = Path(untar_data(URL), force_download=True, path= "/Users/mdurango/Proyect/MLOpsAvanzado/03-model_development")

In [17]:
path.ls()

(#3) [Path('/Users/mdurango/.fastai/data/bdd_simple_1k/images'),Path('/Users/mdurango/.fastai/data/bdd_simple_1k/labels'),Path('/Users/mdurango/.fastai/data/bdd_simple_1k/LICENSE.txt')]

In [14]:
path.ls()
# identificado el path lo muevo a mi carpeta en el repo con mv, fastai es una carpeta oculta por lo que no se ve en el ls
# ls -a muestra las carpetas ocultas


(#3) [Path('/Users/mdurango/.fastai/data/bdd_simple_1k/images'),Path('/Users/mdurango/.fastai/data/bdd_simple_1k/labels'),Path('/Users/mdurango/.fastai/data/bdd_simple_1k/LICENSE.txt')]

In [21]:

def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

In [22]:
def get_classes_per_image(mask_data, class_labels):
    """this function return a dictionary with the presence of each class in the image

    Args:
        - mask_data: numpy array with the mask data
        - class_labels: dictionary with the class labels

    Returns:
        - result_dict: dictionary with the presence of each class in the image
        where 1 means that the class is present and 0 means that the class is not present
    """
    unique = list(np.unique(mask_data))  # unique labels present in the mask image
    result_dict = {}
    for _class in class_labels.keys():
        result_dict[class_labels[_class]] = int(_class in unique)
    return result_dict

In [36]:
def _create_table(image_files, class_labels):
    """ this function creates a data table with the dataset
    Args:
        - image_files: list with the path of the image files
        - class_labels: dictionary with the class labels
    Returns:
        - table: wandb table with the following columns: "File_Name", "Images", "Split" and the class labels""" 
    # Función para crear una tabla de datos con el conjunto de datos
    labels = [str(class_labels[_lab]) for _lab in list(class_labels)]
    # Crear una tabla con las siguientes columnas: "File_Name", "Images", "Split" y las etiquetas de clase
    table = wandb.Table(columns=["File_Name", "Images", "Split"] + labels)
    
    for i, image_file in progress_bar(enumerate(image_files), total=len(image_files)):
        # Abrir la imagen desde el archivo
        image = Image.open(image_file)
        # Leer los datos de máscara desde un archivo utilizando la función label_func
        mask_data = np.array(Image.open(label_func(image_file)))
        # Obtener las clases presentes en la imagen utilizando la función get_classes_per_image
        class_in_image = get_classes_per_image(mask_data, class_labels)
        
        # Agregar datos a la tabla
        table.add_data(
            str(image_file.name),  # Nombre del archivo de imagen
            wandb.Image(
                    image,
                    masks={
                        "predictions": {
                            "mask_data": mask_data,
                            "class_labels": class_labels,
                        }
                    }
            ),  # Imagen con máscaras para visualización
            "None",  # No tenemos una división de conjunto de datos todavía
            *[class_in_image[_lab] for _lab in labels]  # Clases presentes en la imagen
        )
    
    return table  # Devolver la tabla de datos


In [29]:
# start a new run wandb
run = wandb.init(project="segmentation-mlops", entity=None, job_type="upload")
raw_data_at = wandb.Artifact('bdd_simple_1k', type="raw_data")
# Ojo, acá se debe poner el nombre del proyecto, en name o entity, es como el nombre o el usuario
# con que van a instanciar el init, pueden usar su usuario o una organización como entidad (entity)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dramisauria. Use `wandb login --relogin` to force relogin


In [30]:
raw_data_at.add_file(path/'LICENSE.txt', name='LICENSE.txt')

ArtifactManifestEntry(path='LICENSE.txt', digest='X+6ZFkDOlnKesJCNt20yRg==', ref=None, birth_artifact_id=None, size=1594, extra={}, local_path='/Users/mdurango/Library/Application Support/wandb/artifacts/staging/tmp7ro1neyf')

In [31]:
raw_data_at.add_dir(path/'images', name='images')
raw_data_at.add_dir(path/'labels', name='labels')

wandb: Adding directory to artifact (/Users/mdurango/.fastai/data/bdd_simple_1k/images)... Done. 0.4s
wandb: Adding directory to artifact (/Users/mdurango/.fastai/data/bdd_simple_1k/labels)... Done. 0.4s


In [34]:
image_files = get_image_files(path/"images", recurse=False)
# sample a subset if DEBUG
if DEBUG: image_files = image_files[:10]

In [38]:
BDD_CLASSES = {
    i: c
    for i, c in enumerate(
        [
            "background",
            "road",
            "traffic light",
            "traffic sign",
            "person",
            "vehicle",
            "bicycle",
        ]
    )
}

In [39]:
BDD_CLASSES

{0: 'background',
 1: 'road',
 2: 'traffic light',
 3: 'traffic sign',
 4: 'person',
 5: 'vehicle',
 6: 'bicycle'}

In [40]:
table = _create_table(image_files, BDD_CLASSES)

In [41]:
# guardamos la tabla en wandb y terminamos el run
raw_data_at.add(table, "eda_table")

ArtifactManifestEntry(path='eda_table.table.json', digest='Kb95NefVOOU0ulL2SYXFtw==', ref=None, birth_artifact_id=None, size=588824, extra={}, local_path='/Users/mdurango/Library/Application Support/wandb/artifacts/staging/tmpgp_df8zu')

In [42]:
run.log_artifact(raw_data_at)
run.finish()

### Conclusiones de esto:

wandb: Adding directory to artifact (/Users/mdurango/.fastai/data/bdd_simple_1k/images)... Done. 0.4s: Este mensaje indica que se está agregando un directorio al artefacto en wandb. El directorio en cuestión es /Users/mdurango/.fastai/data/bdd_simple_1k/images. El término "Done" significa que la operación se completó con éxito. Además, se muestra el tiempo que tomó la operación, que en este caso fue de 0.4 segundos.

wandb: Adding directory to artifact (/Users/mdurango/.fastai/data/bdd_simple_1k/labels)... Done. 0.4s: Este mensaje es similar al primero, pero se refiere a otro directorio. En este caso, se está agregando el directorio /Users/mdurango/.fastai/data/bdd_simple_1k/labels al artefacto. Al igual que en el primer mensaje, "Done" indica que la operación se realizó con éxito, y se muestra el tiempo que tomó la operación, que también fue de 0.4 segundos.

Estos mensajes son una confirmación de que los directorios especificados se han agregado correctamente como parte del artefacto en wandb. Los artefactos te permiten rastrear y versionar tus datos y recursos, lo que es útil para mantener un registro organizado de los elementos utilizados en tus experimentos y compartirlos con otros colaboradores en wandb si es necesario.


Un artifact puede ser tipo model o bytes. 